# Loads

In [ ]:
!pip install datasets -q
!pip install seqeval -q
!pip install transformers[torch] -q
!pip install flair -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_metric
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from transformers import (
    RobertaTokenizer,
    BertTokenizerFast,
    DataCollatorForTokenClassification,
    RobertaForTokenClassification,
    BertForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline
)

from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from seqeval.metrics import f1_score, classification_report

# BIO Data

В этом разделе делается BIO разметка для текста "Песнь о Нибелунгах" на основе датасета Besnier и Mattingly (2021).

In [ ]:
#считывание текста
with open('Nib_test.txt') as fi:
    text = fi.read()
    splitted_text = text.split('.')

In [ ]:
#разделение на предложения
text_dict = {'id': [], 'text': []}
for i in range(len(splitted_text)):
    clean_text = splitted_text[i].strip().replace("' ", '').replace("'", '').replace("^ ", '').replace("\n ", '') + '.'
    if re.search('[a-zA-Z]', clean_text):
        text_dict['id'].append(i)
        text_dict['text'].append(clean_text)

In [ ]:
#разделение знаков препинания
new_texts = []
for i in text_dict['text']:
    new_i = re.findall(r"[\w']+|[.,!?;]", i)
    new_sent = ' '.join(new_i)
    new_texts.append(new_sent)
text_dict['text'] = new_texts

In [ ]:
text_pd = pd.DataFrame.from_dict(text_dict)
text_pd

,id,text
0,0,I Aventiure von den Nibelungen .
1,1,line tet in alten maeren wunders vil geseit vo...
2,2,"Ez wuohs in Buregonden ein vil edel magedin , ..."
3,3,dar umbe muosen degene vil Verliesen den lip .
4,4,"Ir pflägen dri künige edel unde rich , Gunther..."
...,...,...
5724,5908,die liute heten alle jämer unde nöt .
5725,5909,"mit leide was verendet des küniges höchgezit ,..."
5726,5910,Ine kan iuch niht bescheiden waz sider dä gesc...
5727,5911,Ine sage iu nu niht mere von der grözen not di...


In [ ]:
#считывание части датасета с Nibelungenlied
ner_nib_pd = pd.read_excel('ner-nib.xlsx')
ner_nib_pd

,text,category,lemma,tokens
0,Nibelungenlied,PERSON,Alberich,Albriche:Albrichæ:Albrich:Albrichen:Albriches
1,Nibelungenlied,PERSON,Aldrian,Adrianes:Adrian:Adrianen:Adrians
2,Nibelungenlied,PERSON,Alzey,Alzeye
3,Nibelungenlied,PERSON,Amelrich,Amelrinch:Amelreichen
4,Nibelungenlied,PERSON,Amelungen,Amelunge
...,...,...,...,...
130,Nibelungenlied,PLACE,Thüringen,Durigen
131,Nibelungenlied,PLACE,Ungarn,Vngerlande
132,Nibelungenlied,GROUP,Dänen,Tenen
133,Nibelungenlied,GROUP,Hunnen,Hunin:Hunen


In [ ]:
#список всех токенов из датасета
ner_nib_dict = ner_nib_pd.to_dict('list')
tokens = []
for i in ner_nib_dict['tokens']:
    if type(i) != float:
        new_i = i.split(':')
        tokens.append(new_i)
    else:
        tokens.append([])
ner_nib_dict['tokens'] = tokens

In [ ]:
#два словаря: full_dict, в котором показано, какой именно токен встречается в предложении, а также дана его лемма и тег
#bio_data состоит из слова и его тега для всех предложений, id соответствует номеру предложения
full_dict = {'id': [], 'text': [], 'token': [], 'lemma': [], 'category': []}
bio_data = {'id': [], 'bio_token': [], 'bio_tag': []}
id = 0
flag = False #оба флага показывают, записалось ли слово, то есть есть ли у него тег, если нет, то слову приписывается тег O
bio_flag = False #(один флаг для предложения, другой для конкретного слова)
for sentence in text_dict['text']:
    for word in sentence.split():
        for i in range(len(ner_nib_dict['text'])):
            for token in ner_nib_dict['tokens'][i]:
                if token == word:
                    full_dict['id'].append(id)
                    full_dict['text'].append(sentence)
                    full_dict['token'].append(token)
                    full_dict['lemma'].append(ner_nib_dict['lemma'][i])
                    full_dict['category'].append(ner_nib_dict['category'][i])

                    bio_data['id'].append(id)
                    bio_data['bio_token'].append(word)
                    bio_data['bio_tag'].append('B-'+ ner_nib_dict['category'][i])

                    flag = True
                    bio_flag = True
                    break
                elif ner_nib_dict['lemma'][i] == word.strip():
                    full_dict['id'].append(id)
                    full_dict['text'].append(sentence)
                    full_dict['token'].append(ner_nib_dict['lemma'][i])
                    full_dict['lemma'].append(ner_nib_dict['lemma'][i])
                    full_dict['category'].append(ner_nib_dict['category'][i])

                    bio_data['id'].append(id)
                    bio_data['bio_token'].append(word)
                    bio_data['bio_tag'].append('B-'+ ner_nib_dict['category'][i])

                    flag = True
                    bio_flag = True
                    break
        if bio_flag == False:
            bio_data['id'].append(id)
            bio_data['bio_token'].append(word)
            bio_data['bio_tag'].append('O')
        bio_flag = False

    if flag == False:
        full_dict['id'].append(id)
        full_dict['text'].append(sentence)
        full_dict['token'].append('')
        full_dict['lemma'].append('')
        full_dict['category'].append('')

    id += 1
    flag = False

In [ ]:
full_pd = pd.DataFrame.from_dict(full_dict)
full_pd.head(20)

,id,text,token,lemma,category
0,0,I Aventiure von den Nibelungen .,Nibelungen,Nibelung,PERSON
1,1,line tet in alten maeren wunders vil geseit vo...,,,
2,2,"Ez wuohs in Buregonden ein vil edel magedin , ...",,,
3,3,dar umbe muosen degene vil Verliesen den lip .,,,
4,4,"Ir pflägen dri künige edel unde rich , Gunther...",Gunther,Gunther,PERSON
5,4,"Ir pflägen dri künige edel unde rich , Gunther...",Giselher,Giselher,PERSON
6,5,"diu frouwe was ir swester , die helde hetens i...",,,
7,6,Ein richiu küniginne frou Uote ir muoter hiez .,,,
8,7,"ir vater der hiez Dancrät , der in diu erbe li...",,,
9,8,"Die herren wären milte , von arde hoch erborn ...",,,


In [ ]:
bio_data_pd = pd.DataFrame.from_dict(bio_data)
bio_data_pd.head(20)

,id,bio_token,bio_tag
0,0,I,O
1,0,Aventiure,O
2,0,von,O
3,0,den,O
4,0,Nibelungen,B-PERSON
5,0,.,O
6,1,line,O
7,1,tet,O
8,1,in,O
9,1,alten,O


In [ ]:
#в отдельный файл записывается полная BIO разметка
bio_data_pd.to_csv('bio_nib_full.csv', index=False)

Дальше отсортировываются только те предложения, в которых есть тег B.

In [ ]:
#для каждого id предложения составляется список тегов, которые в него входят
tags_dict = {}
for i in range(len(list(bio_data['id']))):
    if bio_data['id'][i] not in tags_dict.keys():
        tags_dict[bio_data['id'][i]] = []
        tags_dict[bio_data['id'][i]].append(bio_data['bio_tag'][i])
    else:
        tags_dict[bio_data['id'][i]].append(bio_data['bio_tag'][i])

In [ ]:
#список id предложений, в которых есть теги помимо O
new_ids = []
for key, value in tags_dict.items():
    if not len(set(tags_dict[key])) == 1:
        new_ids.append(key)

In [ ]:
#словарь аналогичный bio_data, только с предложениями, в которых есть теги помимо O
new_bio_data_dict = {'id': [], 'bio_token': [], 'bio_tag': []}
for i in range(len(list(bio_data['id']))):
    if bio_data['id'][i] in new_ids:
        new_bio_data_dict['id'].append(bio_data['id'][i])
        new_bio_data_dict['bio_token'].append(bio_data['bio_token'][i])
        new_bio_data_dict['bio_tag'].append(bio_data['bio_tag'][i])

In [ ]:
new_bio_data_pd = pd.DataFrame.from_dict(new_bio_data_dict)
new_bio_data_pd.head(20)

,id,bio_token,bio_tag
0,0,I,O
1,0,Aventiure,O
2,0,von,O
3,0,den,O
4,0,Nibelungen,B-PERSON
5,0,.,O
6,4,Ir,O
7,4,pflägen,O
8,4,dri,O
9,4,künige,O


In [ ]:
#запись его в файл
new_bio_data_pd.to_csv('bio_nib_only_tags.csv', index=False)

# Models

## Data

In [ ]:
#считывание данных
bio_data = pd.read_csv('bio_nib_only_tags.csv')

In [ ]:
#разделение на токены и теги
bio_tokens = bio_data.groupby('id')['bio_token'].agg(list)
bio_tags = bio_data.groupby('id')['bio_tag'].agg(list)

In [ ]:
#разделение на train, validation и test
train_bio_tokens, test_bio_tokens, train_bio_tags, test_bio_tags = train_test_split(bio_tokens, bio_tags, test_size=0.2, random_state=42)
train_bio_tokens, val_bio_tokens, train_bio_tags, val_bio_tags = train_test_split(train_bio_tokens, train_bio_tags, test_size=0.3, random_state=42)

## Functions and classes

In [ ]:
#функция для извлечения пар токен-тег
def get_tokens_pairs(texts, tags, bio=True):

    res_tokens = []
    res_tags = []

    for text, text_tags in zip(texts, tags):
        result = [('[CLS]', 'O')]
        all_tokens = ['[CLS]']
        all_tags = ['O']

        for (word, tag) in zip(text, text_tags):
            tokens = tokenizer.tokenize(word)
            all_tokens.extend(tokens)

            if bio:
                if tag.startswith('B'):
                    all_tags.append(tag)
                    all_tags.extend([tag]*(len(tokens)-1))
                else:
                    all_tags.extend([tag]*len(tokens))
            else:
                if tag == 'O':
                    all_tags.extend([tag]*len(tokens))
                else:
                    all_tags.extend([tag[2:]]*len(tokens))

        all_tokens.append('[SEP]')
        all_tags.append('O')

        res_tokens.append(all_tokens)
        res_tags.append(all_tags)

    return res_tokens, res_tags

In [ ]:
#словари, переводящие тег в цифровое значение, и наоборот
tag2id = {'B-PERSON': 0, 'B-PLACE': 1, 'B-GROUP': 2, 'O': 3}

id2tag = {idx: tag for tag, idx in tag2id.items()}

In [ ]:
#класс для перевода NER датасета в нужный для обучения модели формат
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels, label2id):
        self.tokens = tokens
        self.labels = labels
        self.label2id = label2id

    def prepare_data(self, tokens, labels):
        tokens = torch.tensor([tokenizer.convert_tokens_to_ids(text) for text in tokens])
        labels = torch.tensor([self.label2id[l] for l in labels])
        return tokens, labels

    def __getitem__(self, idx):
        tokens, labels = self.prepare_data(self.tokens[idx], self.labels[idx])
        return {'input_ids': tokens, 'labels': labels}

    def __len__(self):
        return len(self.tokens)

In [ ]:
#загрузка метрик для оценки
metric = load_metric("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[id2tag[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

<ipython-input-10-1c3f9ca0fced>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


## Distilroberta-base-mhg-charter-mlm

### Train

In [ ]:
#определение модели, токенайзера и data collator
model_name = 'atzenhofer/distilroberta-base-mhg-charter-mlm'
tokenizer = RobertaTokenizer.from_pretrained(model_name, model_max_length=512)
data_collator = DataCollatorForTokenClassification(tokenizer)

tokenizer_config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [ ]:
#пары токен-тег для train и validation
train_tokens, train_tags = get_tokens_pairs(train_bio_tokens, train_bio_tags)
val_tokens, val_tags = get_tokens_pairs(val_bio_tokens, val_bio_tags)

In [ ]:
#приводим в нужный формат
train_dataset = NERDataset(train_tokens, train_tags, tag2id)
val_dataset = NERDataset(val_tokens, val_tags, tag2id)

In [ ]:
#идентификация модели
model = RobertaForTokenClassification.from_pretrained(
    model_name,
    num_labels = len(tag2id),
    output_attentions = False,
    output_hidden_states = False
)

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at atzenhofer/distilroberta-base-mhg-charter-mlm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#задаем гиперпараметры и другую дополнительную информацию
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/4 курс/Диплом/models/output_dirs/distilroberta-base-mhg-charter-mlm-run20epoch',
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no'
)

#инициализируем trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

#обучение
trainer.train()

#сохранение
trainer.save_model('/content/drive/MyDrive/4 курс/Диплом/models/distilroberta-base-mhg-charter-mlm-20epoch')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.037646,0.930380,0.945675,0.937965,0.990181
2,No log,0.028305,0.958778,0.964260,0.961511,0.993779
3,0.066500,0.028909,0.943554,0.967834,0.955540,0.993169
4,0.066500,0.036635,0.951782,0.973553,0.962544,0.993962
5,0.010400,0.038681,0.941949,0.974267,0.957836,0.993413
6,0.010400,0.036908,0.958042,0.979271,0.968540,0.994816
7,0.004200,0.035108,0.964739,0.977841,0.971246,0.995243
8,0.004200,0.033969,0.977666,0.969979,0.973807,0.995670
9,0.004200,0.037843,0.964739,0.977841,0.971246,0.995182
10,0.002500,0.038988,0.959689,0.969979,0.964806,0.994450


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Test

In [ ]:
test_tokens, test_tags = get_tokens_pairs(test_bio_tokens, test_bio_tags)

In [ ]:
#проверка на test
classifier = pipeline("ner", model='/content/drive/MyDrive/4 курс/Диплом/models/distilroberta-base-mhg-charter-mlm-20epoch')

pred_labels = []
true_labels = []
for i in tqdm(range(len(test_bio_tokens))):
    res = []
    for word in test_bio_tokens.iloc[i]:
        r = classifier(word)
        res.extend(r)
    preds = []
    for word in res:
        preds.append(id2tag[int(word['entity'].split('_')[-1])])
    pred_labels.append(preds)

for i in tqdm(range(len(test_tokens))):
    true_labels.append(test_tags[i][1:-1])

res_dict = classification_report(true_labels, pred_labels, output_dict=True)

pd.DataFrame(res_dict).T

100%|██████████| 309/309 [00:00<00:00, 491296.41it/s]
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
GROUP,0.000000,0.000000,0.000000,2.0
PERSON,0.911817,0.999034,0.953435,1035.0
PLACE,0.950000,0.883721,0.915663,129.0
micro avg,0.915470,0.984563,0.948760,1166.0
macro avg,0.620606,0.627585,0.623032,1166.0
weighted avg,0.914477,0.984563,0.947620,1166.0


## GHisBERT

### Train

In [ ]:
#определение модели, токенайзера и data collator
model_name = 'christinbeck/GHisBERT'
tokenizer = BertTokenizerFast.from_pretrained(model_name, model_max_length=512)
data_collator = DataCollatorForTokenClassification(tokenizer)

tokenizer_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/219k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

In [ ]:
#пары токен-тег для train и validation
train_tokens, train_tags = get_tokens_pairs(train_bio_tokens, train_bio_tags)
val_tokens, val_tags = get_tokens_pairs(val_bio_tokens, val_bio_tags)

In [ ]:
#приводим в нужный формат
train_dataset = NERDataset(train_tokens, train_tags, tag2id)
val_dataset = NERDataset(val_tokens, val_tags, tag2id)

In [ ]:
#идентификация модели
model = BertForTokenClassification.from_pretrained(
    model_name,
    num_labels = len(tag2id),
    output_attentions = False,
    output_hidden_states = False
)

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at christinbeck/GHisBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#задаем гиперпараметры и другую дополнительную информацию
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/4 курс/Диплом/models/output_dirs/GHisBERT-run20epoch',
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no'
)

#инициализируем trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

#обучение
trainer.train()

#сохранение
trainer.save_model('/content/drive/MyDrive/4 курс/Диплом/models/GHisBERT-20epoch')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.072637,0.872051,0.879231,0.875626,0.980270
2,No log,0.050768,0.973108,0.893870,0.931807,0.988518
3,0.089500,0.050008,0.935574,0.916743,0.926063,0.988033
4,0.089500,0.042720,0.955556,0.944190,0.949839,0.991752
5,0.022800,0.042432,0.964218,0.936871,0.950348,0.991671
6,0.022800,0.040082,0.971510,0.935956,0.953402,0.992318
7,0.010900,0.052383,0.933929,0.956999,0.945323,0.990539
8,0.010900,0.049541,0.975191,0.935041,0.954694,0.992399
9,0.010900,0.051200,0.914387,0.947850,0.930818,0.988437
10,0.007900,0.040848,0.947795,0.963403,0.955535,0.992480


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Test

In [ ]:
test_tokens, test_tags = get_tokens_pairs(test_bio_tokens, test_bio_tags)

In [ ]:
#проверка на test
classifier = pipeline("ner", model='/content/drive/MyDrive/4 курс/Диплом/models/GHisBERT-20epoch')

pred_labels = []
true_labels = []
for i in tqdm(range(len(test_bio_tokens))):
    res = classifier(' '.join(test_bio_tokens.iloc[i]))
    preds = []
    for word in res:
        preds.append(id2tag[int(word['entity'].split('_')[-1])])
    pred_labels.append(preds)

for i in tqdm(range(len(test_tokens))):
    true_labels.append(test_tags[i][1:-1])

res_dict = classification_report(true_labels, pred_labels, output_dict=True)

pd.DataFrame(res_dict).T

100%|██████████| 309/309 [00:00<00:00, 549402.26it/s]


,precision,recall,f1-score,support
GROUP,0.000000,0.000000,0.000000,2.0
PERSON,0.964286,0.980630,0.972389,826.0
PLACE,0.989362,0.845455,0.911765,110.0
micro avg,0.963714,0.962687,0.963200,938.0
macro avg,0.651216,0.608695,0.628051,938.0
weighted avg,0.965170,0.962687,0.963206,938.0


## Flair

### Data for Flair

In [ ]:
#приводим train, test и validation к формату, необходимому для метода ColumnCorpus в Flair
train_bio_tokens_flat = []
for sent in train_bio_tokens:
    for word in sent:
        train_bio_tokens_flat.append(word)
    train_bio_tokens_flat.append('')

train_bio_tags_flat = []
for sent in train_bio_tags:
    for word in sent:
        train_bio_tags_flat.append(word)
    train_bio_tags_flat.append('')

train = pd.DataFrame({'tokens': train_bio_tokens_flat, 'tags': train_bio_tags_flat})
train.to_csv('/content/train.txt', header=None, index=False, sep=' ')

In [ ]:
test_bio_tokens_flat = []
for sent in test_bio_tokens:
    for word in sent:
        test_bio_tokens_flat.append(word)
    test_bio_tokens_flat.append('')

test_bio_tags_flat = []
for sent in test_bio_tags:
    for word in sent:
        test_bio_tags_flat.append(word)
    test_bio_tags_flat.append('')

test = pd.DataFrame({'tokens': test_bio_tokens_flat, 'tags': test_bio_tags_flat})
test.to_csv('/content/test.txt', header=None, index=False, sep=' ')

In [ ]:
val_bio_tokens_flat = []
for sent in val_bio_tokens:
    for word in sent:
        val_bio_tokens_flat.append(word)
    val_bio_tokens_flat.append('')

val_bio_tags_flat = []
for sent in val_bio_tags:
    for word in sent:
        val_bio_tags_flat.append(word)
    val_bio_tags_flat.append('')

val = pd.DataFrame({'tokens': val_bio_tokens_flat, 'tags': val_bio_tags_flat})
val.to_csv('/content/val.txt', header=None, index=False, sep=' ')

### Train

In [ ]:
#определяем колонки
columns = {0: 'text', 1: 'ner'}

#папка, в которой хранятся train, test и validation
data_folder = '/content'

#определяем корпус
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='val.txt')

2024-05-25 09:05:41,037 Reading data from /content
2024-05-25 09:05:41,046 Train: /content/train.txt
2024-05-25 09:05:41,050 Dev: /content/val.txt
2024-05-25 09:05:41,053 Test: /content/test.txt


In [ ]:
#какую колонку надо предсказывать
label_type = 'ner'

In [ ]:
#словарь из тегов в корпусе
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

2024-05-25 09:05:46,392 Computing label dictionary. Progress:


0it [00:00, ?it/s]
862it [00:00, 12369.49it/s]

2024-05-25 09:05:46,518 Dictionary created for label 'ner' with 3 values: PERSON (seen 938 times), PLACE (seen 161 times), GROUP (seen 1 times)
Dictionary with 3 tags: PERSON, PLACE, GROUP


In [ ]:
#инициализируем Flair эмбеддинги
embedding_types = [
    FlairEmbeddings('de-forward'),
    FlairEmbeddings('de-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

2024-05-25 09:05:48,890 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-mix-german-forward-v0.2rc.pt not found in cache, downloading to /tmp/tmpnhqx_ix4


100%|██████████| 69.4M/69.4M [00:07<00:00, 9.90MB/s]

2024-05-25 09:05:57,198 copying /tmp/tmpnhqx_ix4 to cache at /root/.flair/embeddings/lm-mix-german-forward-v0.2rc.pt
2024-05-25 09:05:57,292 removing temp file /tmp/tmpnhqx_ix4


2024-05-25 09:05:59,139 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-mix-german-backward-v0.2rc.pt not found in cache, downloading to /tmp/tmpf9xwmp2g


100%|██████████| 69.4M/69.4M [00:06<00:00, 10.9MB/s]

2024-05-25 09:06:06,558 copying /tmp/tmpf9xwmp2g to cache at /root/.flair/embeddings/lm-mix-german-backward-v0.2rc.pt


2024-05-25 09:06:06,657 removing temp file /tmp/tmpf9xwmp2g


In [ ]:
#инииализируем SequenceTagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type)

2024-05-25 09:06:06,945 SequenceTagger predicts: Dictionary with 13 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-PLACE, B-PLACE, E-PLACE, I-PLACE, S-GROUP, B-GROUP, E-GROUP, I-GROUP


In [ ]:
#инициализируем trainer
trainer = ModelTrainer(tagger, corpus)

In [ ]:
#обучение
trainer.train('/content/drive/MyDrive/4 курс/Диплом/models/flair-20epoch-1',
              learning_rate=1e-1,
              mini_batch_size=4,
              max_epochs=20)

2024-05-25 09:28:03,885 ----------------------------------------------------------------------------------------------------
2024-05-25 09:28:03,898 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=15, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2024-05-25 09:28:03,903 ------------------------------------

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:84: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2024-05-25 09:28:03,995 ----------------------------------------------------------------------------------------------------
2024-05-25 09:28:04,001 Final evaluation on model from best epoch (best-model.pt)
2024-05-25 09:28:04,009  - metric: "('micro avg', 'f1-score')"
2024-05-25 09:28:04,013 ----------------------------------------------------------------------------------------------------
2024-05-25 09:28:04,014 Computation:
2024-05-25 09:28:04,016  - compute on device: cuda:0
2024-05-25 09:28:04,019  - embedding storage: cpu
2024-05-25 09:28:04,020 ----------------------------------------------------------------------------------------------------
2024-05-25 09:28:04,027 Model training base path: "/content/drive/MyDrive/4 курс/Диплом/models/flair-20epoch-1"
2024-05-25 09:28:04,031 ----------------------------------------------------------------------------------------------------
2024-05-25 09:28:04,036 -------------------------------------------------------------------------------

100%|██████████| 6/6 [00:02<00:00,  2.96it/s]

2024-05-25 09:28:20,940 DEV : loss 0.024281982332468033 - f1-score (micro avg)  0.9578
2024-05-25 09:28:20,969  - 0 epochs without improvement
2024-05-25 09:28:20,975  - 0 epochs without improvement
2024-05-25 09:28:20,981  - 0 epochs without improvement
2024-05-25 09:28:20,987  - 0 epochs without improvement
2024-05-25 09:28:20,992  - 0 epochs without improvement
2024-05-25 09:28:20,998 saving best model


...

2024-05-25 09:31:29,862 DEV : loss 0.018185140565037727 - f1-score (micro avg)  0.9688
2024-05-25 09:31:29,890  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.0015625]
2024-05-25 09:31:29,895  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.00078125]
2024-05-25 09:31:29,900  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.000390625]
2024-05-25 09:31:29,906  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.0001953125]
2024-05-25 09:31:29,912  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [9.765625e-05]
2024-05-25 09:31:29,914 ----------------------------------------------------------------------------------------------------
2024-05-25 09:31:29,916 learning rate too small - quitting training!
2024-05-25 09:31:29,917 ----------------------------------------------------------------------------------------------------
2024-

2024-05-25 09:31:30,774 Done.
2024-05-25 09:31:30,775 ----------------------------------------------------------------------------------------------------
2024-05-25 09:31:30,778 Loading model from best epoch ...
2024-05-25 09:31:31,825 SequenceTagger predicts: Dictionary with 15 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-PLACE, B-PLACE, E-PLACE, I-PLACE, S-GROUP, B-GROUP, E-GROUP, I-GROUP, <START>, <STOP>


100%|██████████| 5/5 [00:03<00:00,  1.54it/s]

2024-05-25 09:31:35,397 
Results:
- F-score (micro) 0.9631
- F-score (macro) 0.6324
- Accuracy 0.931

By class:
              precision    recall  f1-score   support

      PERSON     0.9737    0.9652    0.9694       345
       PLACE     0.9783    0.8824    0.9278        51
       GROUP     0.0000    0.0000    0.0000         1

   micro avg     0.9742    0.9521    0.9631       397
   macro avg     0.6506    0.6159    0.6324       397
weighted avg     0.9718    0.9521    0.9616       397

2024-05-25 09:31:35,399 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9630573248407643}